In [1]:
import random

import numpy as np
import pandas as pd

import torch

torch.cuda.empty_cache()

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

In [2]:
# Фиксируем random seed
SEED = 42
seed_all(SEED)

In [3]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [4]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [5]:
from app.models.translate_model import TranslationModel, TranslationModelCT2

In [6]:
from pathlib import Path
import json

long_text_path = Path("long_text.json")

with open(long_text_path, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = data["texts"]
source_lang = data["source_lang"]
target_langs = data["target_langs"]
data

{'texts': ['Глава 1. Призыв к битве\n\nМир древности был полон чудес и чудовищ. В те дни на земле великих правителей и легендарных героев, сражения не ограничивались только оружием и армиями. Они были испытаниями силы духа, мужества и ума. Одним из таких испытаний стали гонки на колесницах, что стало невероятно популярным в различных уголках мира, и даже в Иудее, где собрались великие герои Библии.\n\nЭти гонки, однако, не были простыми. Они не были лишь забава ради. Каждая колесница олицетворяла не только военную мощь, но и внутренний бой, борьбу с судьбой, и даже божественные вмешательства. Колесницы, запряженные сильными животными, мчались по древним трассам, где каждое движение было решающим. Гонщики, ставшие участниками этих состязаний, были героями, чьи имена известны всем — Моисей, Давид, Илия и многие другие.\n\nГлава 2. Моисей и его колесница\n\nМоисей, великий вождь и пророк, был на старости лет, но в его глазах горел огонь непокорного духа. Он решил принять участие в этих го

## Квантованная модель

In [ ]:
model = TranslationModelCT2(
    model_name="michaelfeil/ct2fast-m2m100_418M",    
    # model_name="michaelfeil/ct2fast-m2m100_1.2B",
    # model_name="michaelfeil/ct2fast-m2m100-12B-last-ckpt",
    
    # model_name="entai2965/m2m100-418M-ctranslate2",
    # model_name="entai2965/m2m100-1.2B-ctranslate2",
    
    cache_dir=DATA_CACHE,
    device="cpu"
)

In [8]:
generation_kwargs = {
    "beam_size": 5,
    # "replace_unknowns": True,
    "no_repeat_ngram_size": 3
}


In [9]:
# Переводим тексты
translations = model.translate_batch(
    texts=texts,
    source_lang=source_lang,
    target_langs=target_langs,
    **generation_kwargs
)

# Выводим результат
for lang, translated_list in translations.items():
    print(f"Переводы на язык '{lang}':")
    for idx, translated_text in enumerate(translated_list):
        print(f"  [{idx + 1}] {translated_text}")
    print()

Переводы на язык 'en':
  [1] The Chapter 1. The world of antiquity was full of wonders and monsters. In those days on the land of great rulers and legendary heroes, battles were not limited to weapons and armies. They were tests of spirit strength, courage and intelligence. One such test was the chariot race, which became incredibly popular in various parts of the world, and even in Judea, where the great heroes of the Bible gathered. These races, however, were not easy. They were not just for fun. Each chariot represented not only military power, but also internal battle, struggle with fate, and even divine interventions. Chariots, trailed by strong animals, raced on ancient trails where every movement was decisive. The riders who participated in these competitions were heroes whose names are known to everyone – Moses, David, Elijah and many others. The Chapter 2. Moses and his chariot Moses, the great chief and prophet, was in old age, but in his eyes burned the fire of the disobedie

## Обычная модель

In [10]:
model = TranslationModel(
    model_name="facebook/m2m100_418M",
    # model_name="facebook/m2m100_1.2B",
    cache_dir=DATA_CACHE,
    device="cpu"
)

In [11]:
# Переводим тексты
translations = model.translate_batch(
    texts=texts,
    source_lang=source_lang,
    target_langs=target_langs
)

# Выводим результат
for lang, translated_list in translations.items():
    print(f"Переводы на язык '{lang}':")
    for idx, translated_text in enumerate(translated_list):
        print(f"  [{idx + 1}] {translated_text}")
    print()

d:\Projects\env_ml\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Переводы на язык 'en':
  [1] Chapter 1 The call to the battle The world of ancient times was full of miracles and monsters. In those days on the ground of great rulers and legendary heroes, the battles were not limited to weapons and armies. They were the trials of the strength of spirit, courage and mind. One of these trials was the races on the wheels, which became incredibly popular in different corners of the world, and even in Judea, where the great heroes of the Bible gathered. However, these races were not easy. They were not just fun. Each wheel represented not only the military power, but also the internal battle, the fate struggle, and even the divine interventions. The wheels, stretched by strong animals, walked on ancient trails, where every movement was decisive. The runners who participated in these competitions were heroes whose names are known to everyone—Moses, David, Elijah and many others. Chapter 2 . Moses and his wardrobe Moses, the great leader and prophet, were o